In [1]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

## Generate output

In [2]:
def power_law(x, a, b):
    return a * np.power(x, b)

def exponential(x, a, b):
    return a * np.exp(b * x)

def stretched_exponential(x, a, b, c):
    return a * np.exp(-np.power(b * x, c))

In [3]:
def calc_params(df, shape):
    trv = df[df['shape']==shape]['trv']
    ct, bins = np.histogram(trv, bins=np.arange(0, np.ceil(df['trv'].max())), density=False)
    cum = np.cumsum(ct[::-1])
    rev_bin = bins[::-1]
    x = rev_bin[:-1]
    y = cum
    params_power, _ = curve_fit(power_law, x, y, p0=[1, -1])
    params_exp, _ = curve_fit(exponential, x, y, p0=[1, 0.01])
    params_stretch, _ = curve_fit(stretched_exponential, x, y, p0=[1, 0.01, 1], 
                                bounds=([0, 0, 0], [np.inf, np.inf, 10]), maxfev=5000)
    x_fit = np.linspace(min(x), max(x), 500)
    y_power_fit = power_law(x_fit, *params_power)
    y_exp_fit = exponential(x_fit, *params_exp)
    y_stretch_fit = stretched_exponential(x_fit, *params_stretch)
    y_power_pred = power_law(x, *params_power)
    y_exp_pred = exponential(x, *params_exp)
    y_stretch_pred = stretched_exponential(x, *params_stretch)
    r2_power = r2_score(y, y_power_pred)
    r2_exp = r2_score(y, y_exp_pred)
    r2_stretch = r2_score(y, y_stretch_pred)
    # plt.scatter(x, y, label="Data", color="black", s=10)
    # plt.plot(x_fit, y_power_fit, label=f"Power Law: $R^2$ = {r2_power:.3f}", linewidth=2)
    # plt.plot(x_fit, y_exp_fit, label=f"Exponential: $R^2$ = {r2_exp:.3f}", linewidth=2, linestyle="--")
    # plt.plot(x_fit, y_stretch_fit, label=f"Stretched Exp.: $R^2$ = {r2_stretch:.3f}", linewidth=2, linestyle=":")
    # plt.xlabel("x")
    # plt.ylabel("y")
    # plt.title(f"{f}: {len(trv)} trials")
    # plt.legend()
    # plt.savefig(f"../fit/{f}.png")
    # plt.close()
    # with open(f"../fit/{f}.params", 'w') as r:
    #     r.write(f"Power Law: {params_power}\n")
    #     r.write(f"Exponential: {params_exp}\n")
    #     r.write(f"Stretched Exponential: {params_stretch}\n")
    return params_exp



In [4]:
def bootstrapping_results(df, shape):
    n = len(df)
    bs_a = np.zeros(n)
    bs_b = np.zeros(n)
    for i in range(n):
        bs_sample = df.iloc[np.random.randint(n, size=n)]
        params = calc_params(bs_sample, shape)
        bs_a[i] = params[0]
        bs_b[i] = params[1] 
    lambda_ = -1/bs_b
    avg = lambda_.mean()
    std = lambda_.std()
    return avg, std, lambda_
    

In [5]:
def bootstrapping_exp(df, shape):
    n = len(df)
    bs_a = np.zeros(n)
    bs_b = np.zeros(n)
    for i in range(n):
        bs_sample = df.iloc[np.random.randint(n, size=n)]
        params = calc_params(bs_sample, shape)
        bs_a[i] = params[0]
        bs_b[i] = params[1] 
    lambda_ = -1/bs_b
    avg = lambda_.mean()
    std = lambda_.std()
    return avg, std

In [8]:
dis_dir = ['../cur0.71_done/analysis/distance.csv', 
           '../cur0.71_fixbead/analysis/distance.csv',
           '../cur0.71_cnst2200/analysis/distance.csv',
           '../cur0.71_cnst220000/analysis/distance.csv',
           '../cur0.71_cnst220/analysis/distance.csv']

In [7]:
dis_dir[0].split('/')[1]+'_cur.csv'

'cur0.71_done_cur.csv'

In [9]:
bac_length = 4
lambda_df = pd.DataFrame()
for d in dis_dir:
    df = pd.read_csv(d)
    def _safe_diam(name):
        parts = str(name).split('_')
        try:
            return float(parts[2])
        except (IndexError, ValueError):
            return np.nan
    df['diam'] = [ _safe_diam(i) for i in df['name']]
    df['ete'] = df['ete']/df['diam']/bac_length
    df['trv'] = df['trv']/df['diam']/bac_length
    df_cur = df.loc[df['shape']=='cur', :]
    df_str = df.loc[df['shape']=='str', :]
    prefix = d.split('/')[1] if len(d.split('/')) > 1 else os.path.basename(d)
    cur_col = f"{prefix}_cur"
    str_col = f"{prefix}_str"
    if len(df_cur) > 0:
        avg_cur, std_cur, lambda_cur = bootstrapping_results(df_cur, 'cur')
        lambda_df[cur_col] = pd.Series(lambda_cur)
    else:
        avg_cur = std_cur = np.nan
        lambda_df[cur_col] = pd.Series(dtype=float)
    if len(df_str) > 0:
        avg_str, std_str, lambda_str = bootstrapping_results(df_str, 'str')
        lambda_df[str_col] = pd.Series(lambda_str)
    else:
        avg_str = std_str = np.nan
        lambda_df[str_col] = pd.Series(dtype=float)
    with open('bootstrapping_n.txt', 'a') as f:
        f.write(f"{d}, {avg_cur}, {std_cur}, {avg_str}, {std_str}\n")
# (optional) save lambdas to CSV
# lambda_df.to_csv('bootstrapping_lambdas.csv', index=False)

In [10]:
lambda_df.to_csv('bootstrapping_lambdas.csv', index=False)

## Analyze data

In [11]:
lambda_df = pd.read_csv('./bootstrapping_lambdas.csv')
lambda_df.head()

,cur0.71_done_cur,cur0.71_done_str,cur0.71_fixbead_cur,cur0.71_fixbead_str,cur0.71_cnst2200_cur,cur0.71_cnst2200_str,cur0.71_cnst220000_cur,cur0.71_cnst220000_str,cur0.71_cnst220_cur,cur0.71_cnst220_str
0,27.672685,21.156092,25.524093,18.991416,29.139036,20.545416,25.229461,19.503066,43.784369,28.689689
1,26.517189,22.918165,24.998704,18.267185,30.289950,19.939024,28.688186,19.116286,45.459234,29.745767
2,26.148227,21.056475,26.278389,18.785999,31.802382,22.070971,26.405785,18.982823,44.215643,28.480593
3,26.666274,21.018278,27.924511,19.036426,30.616489,23.135705,25.760903,20.376828,44.356165,29.257958
4,28.501835,21.918992,26.423527,19.146677,31.004802,20.953441,25.737622,20.542073,39.363123,29.487853


In [12]:
lambda_df.columns

Index(['cur0.71_done_cur', 'cur0.71_done_str', 'cur0.71_fixbead_cur',
       'cur0.71_fixbead_str', 'cur0.71_cnst2200_cur', 'cur0.71_cnst2200_str',
       'cur0.71_cnst220000_cur', 'cur0.71_cnst220000_str',
       'cur0.71_cnst220_cur', 'cur0.71_cnst220_str'],
      dtype='object')

In [4]:
# choose columns (example uses 'str')
a = lambda_df['cur0.71_done_cur'].values
b = lambda_df['cur0.71_done_str'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(cur) < lambda(str)) ≈", p_value)

P(lambda(cur) < lambda(str)) ≈ 0.0


In [13]:
a = lambda_df['cur0.71_cnst220_cur'].values
b = lambda_df['cur0.71_cnst2200_cur'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(220) < lambda(2200)) ≈", p_value)

P(lambda(220) < lambda(2200)) ≈ 0.0


In [14]:
a = lambda_df['cur0.71_cnst220_str'].values
b = lambda_df['cur0.71_cnst2200_str'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(220) < lambda(2200)) ≈", p_value)

P(lambda(220) < lambda(2200)) ≈ 0.0


In [ ]:
# choose columns
a = lambda_df['cur0.71_cnst2200_cur'].values
b = lambda_df['cur0.71_done_cur'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(2200) < lambda(22000)) ≈", p_value)

P(lambda(2200) < lambda(22000)) ≈ 0.00679


In [27]:
a = lambda_df['cur0.71_done_cur'].values
b = lambda_df['cur0.71_cnst220000_cur'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(22000) < lambda(220000)) ≈", p_value)

P(lambda(22000) < lambda(220000)) ≈ 0.28338


In [33]:
a = lambda_df['cur0.71_cnst220000_cur'].values
b = lambda_df['cur0.71_fixbead_cur'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(220000) < lambda(fixed)) ≈", p_value)

P(lambda(220000) < lambda(fixed)) ≈ 0.50299


In [28]:
# choose columns (example uses 'str')
a = lambda_df['cur0.71_cnst2200_str'].values
b = lambda_df['cur0.71_done_str'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(2200) < lambda(22000)) ≈", p_value)

P(lambda(2200) < lambda(22000)) ≈ 0.54173


In [29]:
a = lambda_df['cur0.71_done_str'].values
b = lambda_df['cur0.71_cnst220000_str'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(22000) < lambda(220000)) ≈", p_value)

P(lambda(22000) < lambda(220000)) ≈ 0.03652


In [32]:
a = lambda_df['cur0.71_cnst220000_str'].values
b = lambda_df['cur0.71_fixbead_str'].values

n_trials = 100000
i = np.random.randint(len(a), size=n_trials)
j = np.random.randint(len(b), size=n_trials)
p_value = np.mean(a[i] < b[j])
print("P(lambda(220000) < lambda(fixed)) ≈", p_value)

P(lambda(220000) < lambda(fixed)) ≈ 0.26595


In [ ]:
# bac_length = 4

# for d in dis_dir:
#     df = pd.read_csv(d)
#     df['diam'] = [float(i.split('_')[2]) for i in df['name']]
#     df['ete'] = df['ete']/df['diam']/bac_length
#     df['trv'] = df['trv']/df['diam']/bac_length
#     df_cur = df.loc[df['shape']=='cur', :]
#     df_str = df.loc[df['shape']=='str', :]
#     avg_cur, std_cur = bootstrapping_exp(df_cur, 'cur')
#     avg_str, std_str = bootstrapping_exp(df_str, 'str')
#     with open('bootstrapping.txt', 'a') as f:
#         f.write(f"{d}, {avg_cur}, {std_cur}, {avg_str}, {std_str}\n")
#     # print(d, avg_cur, std_cur, avg_str, std_str)

/tmp/tmp.QKlCvLIk6o/ipykernel_281187/1092078451.py:10: RuntimeWarning: divide by zero encountered in divide
  lambda_ = -1/bs_b
/home/ziyangc/.conda/envs/curvature/lib/python3.12/site-packages/numpy/_core/_methods.py:191: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/tmp/tmp.QKlCvLIk6o/ipykernel_281187/1092078451.py:10: RuntimeWarning: divide by zero encountered in divide
  lambda_ = -1/bs_b
/home/ziyangc/.conda/envs/curvature/lib/python3.12/site-packages/numpy/_core/_methods.py:191: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/tmp/tmp.QKlCvLIk6o/ipykernel_281187/1092078451.py:10: RuntimeWarning: divide by zero encountered in divide
  lambda_ = -1/bs_b
/home/ziyangc/.conda/envs/curvature/lib/python3.12/site-packages/numpy/_core/_methods.py:191: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/tmp/tmp.QKlCvLIk6o/ipykernel_281187/1092078451.py:10: RuntimeWarning: divid